In [1]:
import os
import numpy as np 
import pandas as pd
from scipy import stats, signal, io
import mne
from mne import create_info, EpochsArray
from mne.time_frequency import tfr_morlet
from bids import BIDSLayout
import mne_bids
import scipy
#import settings
import json
#from coordinates_io import BIDS_coord
from sklearn import linear_model, metrics
import multiprocessing
from sklearn.model_selection import cross_val_score
from sklearn import ensemble, model_selection
from matplotlib import pyplot as plt
import matplotlib 
from sklearn import ensemble, neural_network, neighbors

In [2]:
out_path = '/home/icn/Documents/raw_out/RF_32_4/'
BIDS_path = '/home/icn/Documents/BIDS/'

In [3]:
subject_exclude = ['002', '003', '011', '012', '015', '016']
subject_tf_usage = ['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014', '016']
#002 - DBS4011 - no locaization possible
#003 - DBS4020 - LEFT due to signal quality
#011 - DBS4037 - STN was not recorded
#012 - DBS4030 - GPi not STN 
#015 - DBS4065 - LEFT due to signal quality

In [4]:
# setup plot where STN and ECOG is visible 
faces = io.loadmat('faces.mat')
Vertices = io.loadmat('Vertices.mat')
grid = io.loadmat('grid.mat')['grid']
stn_surf = io.loadmat('STN_surf.mat')
x_ = stn_surf['vertices'][::2,0]
y_ = stn_surf['vertices'][::2,1]
x_ecog = Vertices['Vertices'][::1,0]
y_ecog = Vertices['Vertices'][::1,1]
x_stn = stn_surf['vertices'][::1,0]
y_stn = stn_surf['vertices'][::1,1]

In [5]:
def get_x_y_c(dict_ch, ind_, Con=True, return_z=False):
    
    channels = list(dict_ch.keys())
    x_choord = np.zeros(len(ind_))
    y_choord = np.zeros(len(ind_))
    z_choord = np.zeros(len(ind_))
    c_ = np.zeros(len(ind_))
    for idx, ind_ch in enumerate(ind_): 
        choord = np.squeeze(np.array(dict_ch[channels[ind_ch]]["choords"]))
        x_choord[idx] = choord[0]
        y_choord[idx] = choord[1]
        z_choord[idx] = choord[2]
        
        #wähle den contralateralen channel aus
        mov_channels = dict_ch[channels[ind_ch]]['mov_ch']

        mov_idx = get_movement_idx(channels[ind_ch], mov_channels, Con)

        c_[idx] = dict_ch[channels[ind_ch]]["res_"+mov_channels[mov_idx]]["R2"]
    if return_z is True:
        return x_choord, y_choord, z_choord, c_
    return x_choord, y_choord, c_

In [6]:
subject_id = []
for patient_test in range(17):
    if patient_test < 10:
        subject_id.append(str('00') + str(patient_test))
    else:
        subject_id.append(str('0') + str(patient_test))


In [7]:
def get_movement_idx(ch, mov_channels, Con=True):
    mov_idx = 0
    if len(mov_channels) > 1:    
        if Con is True:
            if ("RIGHT" in ch and "LEFT" in mov_channels[0]) or \
                ("LEFT" in ch and "RIGHT" in mov_channels[0]):
                mov_idx = 0
            if ("RIGHT" in ch and "LEFT" in mov_channels[1]) or \
                ("LEFT" in ch and "RIGHT" in mov_channels[1]):
                mov_idx = 1
        else:
            if ("RIGHT" in ch and "RIGHT" in mov_channels[0]) or \
                ("LEFT" in ch and "LEFT" in mov_channels[0]):
                mov_idx = 0
            if ("RIGHT" in ch and "RIGHT" in mov_channels[1]) or \
                ("LEFT" in ch and "LEFT" in mov_channels[1]):
                mov_idx = 1
    return mov_idx

### get the data

In [8]:
subject_id_ = '000'

In [9]:
with open(out_path+'sub_'+subject_id_+'_patient_concat.json', 'r') as fp:
        dict_ch = json.load(fp)

In [10]:
channels = list(dict_ch.keys())

In [11]:
channels[0]

'ECOG_RIGHT_0'

In [12]:
for sub_idx, subject_id_ in enumerate(subject_id):    
    if subject_id_ in subject_exclude: 
            continue
    with open(out_path+'sub_'+subject_id_+'_patient_concat.json', 'r') as fp:
        dict_ch = json.load(fp)


In [13]:
dict_ch[channels[0]].keys()

dict_keys(['data', 'true_movements', 'mov_ch', 'choords', 'res_MOV_RIGHT', 'res_MOV_LEFT'])

In [14]:
np.array(dict_ch[channels[0]]['data']).shape

(8, 7380)

In [15]:
np.array(dict_ch[channels[0]]['mov_ch']).shape

(2,)

In [28]:
np.array(dict_ch[channels[0]]['res_MOV_RIGHT'])

array({'R2': 0.40858242561094277}, dtype=object)

In [9]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
from sklearn.model_selection import RandomizedSearchCV# Create the parameter grid based on the results of random search 

param_grid = {
    'bootstrap': [True],
    'max_depth': [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, None],
    'max_features': [1,2,3,5, 11],
    'min_samples_leaf': [1, 2, 4, 6],
    'min_samples_split': [2, 8, 10, 12],
    'n_estimators': [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000]
}# Create a based model
rf = ensemble.RandomForestRegressor()# Instantiate the grid search model
rand_search = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, 
                          cv = 3, n_jobs = 62, verbose = 10, n_iter=100, scoring='neg_mean_absolute_error')

NameError: name 'RandomForestRegressor' is not defined

In [ ]:
# Fit the grid search to the data
rand_search.fit(X, y)

In [ ]:
rand_search.best_estimator_

In [ ]:
rand_search.best_estimator_